# Waston Assistant Customer Effort Analysis Notebook

## Introduction
This notebook demonstrates how to measure and analyze the performance improvement you achieve when enabling Watson Assistant Autolearning feature. We use **Customer Effort** as the evaluation metric which quantifies how much effort users must make in order to achieve their intents.

#### Programming language and environment
Some familiarity with Python is recommended. This notebook runs on Python 3.5+ environment.

## Table of contents
1. [Configuration and setup](#setup)<br>
    1.1 [Import and apply global CSS styles](#css)<br>
    1.2 [Install required Python libraries](#python)<br>
    1.3 [Import functions used in the notebook](#function)<br>
2. [Load and format data](#load)<br>
    2.1 [Option one: from a Watson Assistant instance](#load_remote)<br>
    2.2 [Option two: from JSON files](#load_local)<br>
    2.3 [Format the log data](#format_data)<br>
3. [Visualize Customer Effort](#overview)<br>
    3.1 [Extract utterances containing disambiguations or more options](#conversation_extraction)<br>
    3.2 [Calculate customer effort](#calculate)<br>
    3.3 [Visualize](#visualize)<br>
4. [Analyze Customer Effort](#analysis)<br>
    4.1 [Customer effort vs user clicks](#effort_click)<br>
    4.2 [Top dialog nodes with the highest effort](#effort_high)<br>
    4.3 [Customer effort on dialog nodes](#effort_node)<br>
    4.4 [Customer effort on utterances](#effort_utterance)<br>
5. [Identify the most confused dialog nodes](#node)<br>
    5.1 [Top N frequently co-occurred dialog nodes](#node_top)<br>
    5.2 [Dialog node co-occurrence heatmap](#node_heat)<br>
6. [Analyze user clicks](#click)<br>
<!-- 6. [Summary and next steps](#summary)<br> --> <a id="analysis"></a>

<a id="setup"></a>
## 1. Configuration and Setup

In this section, we import required libraries and functions.

### <a id="css"></a> 1.1 Import and apply global CSS styles

In [1]:
from IPython.display import HTML
!curl -O https://raw.githubusercontent.com/watson-developer-cloud/assistant-improve-recommendations-notebook/master/src/main/css/custom.css
HTML(open('custom.css', 'r').read())

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   348  100   348    0     0    348      0  0:00:01 --:--:--  0:00:01  1487


### <a id="python"></a> 1.2 Install required Python libraries

In [2]:
# After running this cell once, comment out the following code. Packages only need to be installed once.
!pip3 install --user --upgrade "ibm-watson==4.1.0";
!pip3 install --user --upgrade "bokeh==2.0.0";
!pip3 install --user --upgrade "pandas==1.0.1";
!pip3 install --user --upgrade "tqdm==4.43.0";

# Import required libraries
import pandas as pd
import json
from ibm_watson import AssistantV1
from IPython.display import display
import numpy as np
import re


Requirement already up-to-date: ibm-watson==4.1.0 in /Users/zhezhang/.local/lib/python3.6/site-packages (4.1.0)
You are using pip version 18.1, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Requirement already up-to-date: bokeh==2.0.0 in /Users/zhezhang/.local/lib/python3.6/site-packages (2.0.0)
You are using pip version 18.1, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Requirement already up-to-date: pandas==1.0.1 in /Users/zhezhang/.local/lib/python3.6/site-packages (1.0.1)
You are using pip version 18.1, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Requirement already up-to-date: tqdm==4.43.0 in /Users/zhezhang/.local/lib/python3.6/site-packages (4.43.0)
You are using pip version 18.1, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip

### <a id="function"></a> 1.3 Import functions used in the notebook

In [3]:
# Import Watson Assistant related functions
from ibm_cloud_sdk_core.authenticators import BasicAuthenticator
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

!curl -O https://raw.githubusercontent.com/watson-developer-cloud/assistant-improve-recommendations-notebook/library_al/src/main/python/watson_assistant_func.py
!curl -O https://raw.githubusercontent.com/watson-developer-cloud/assistant-improve-recommendations-notebook/library_al/src/main/python/visualize_func.py
!curl -O https://raw.githubusercontent.com/watson-developer-cloud/assistant-improve-recommendations-notebook/library_al/src/main/python/computation_func.py
from watson_assistant_func import get_logs, get_assistant_definition
from computation_func import format_logs_disambiguation, calculate_effort, extract_disambiguation_utterances, generate_cooccurrence_matrix
from visualize_func import show_effort_over_time, show_node_effort, show_top_node_effort, show_input_effort, show_disambiguation_click, show_more_options_click, show_cooccured_nodes, show_cooccured_heatmap, show_click_vs_effort
    

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10381  100 10381    0     0  10381      0  0:00:01 --:--:--  0:00:01 50393
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 95481  100 95481    0     0  95481      0  0:00:01 --:--:--  0:00:01  412k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 45063  100 45063    0     0  45063      0  0:00:01 --:--:--  0:00:01  193k


## <a id="load"></a> 2. Load and format data 

### <a id="load_remote"></a> 2.1 Option one: from a Watson Assistant instance

#### 2.1.1 Add Watson Assistant configuration

This notebook uses the Watson Assistant v1 API to access your skill definition and your logs. Provide your Watson Assistant credentials and the workspace id that you want to fetch data from.

You can access the values you need for this configuration from the Watson Assistant user interface. Go to the Skills page and select View API Details from the menu of a skill tile.

- The string to set in the call to `IAMAuthenticator` is your Api Key under Service Credentials
- The string to set for version is a date in the format version=YYYY-MM-DD. The version date string determines which version of the Watson Assistant V1 API will be called. For more information about version, see [Versioning](https://cloud.ibm.com/apidocs/assistant/assistant-v1#versioning).
- The string to pass into `assistant.set_service_url` is the portion of the Legacy v1 Workspace URL that ends with `/api`. For example, `https://gateway.watsonplatform.net/assistant/api`. This value will be different depending on the location of your service instance. Do not pass in the entire Workspace URL.

In [4]:
# Provide credentials to connect to assistant
authenticator = BasicAuthenticator('apikey', 'API_KEY')
sdk_object = AssistantV1(version='2020-02-05', authenticator=authenticator)
sdk_object.set_service_url('URL')

Specify your assistant information.

In [5]:
assistant_information = {'workspace_id' : '',
                         'skill_id' : '',
                         'assistant_id' : ''}

#### 2.1.2 Fetch and load your assistant definition

Fetch assistant definition and load into a dataframe. Note that assistant definition will be saved into a cached file and reloaded from the file. Set `overwrite` to True to reload assistant definition.

In [ ]:
df_assistant = get_assistant_definition(sdk_object, assistant_information, overwrite=True)

# Get all intents
assistant_intents = [intent['intent'] for intent in df_assistant['intents'].values[0]] 

# Get all dialog nodes
assistant_nodes = pd.DataFrame(df_assistant['dialog_nodes'].values[0])

if df_assistant is not None:
    assistant_loaded = True
else:
    assistant_loaded = False

#### 2.1.3 Fetch and load logs

Fetch user generated logs. You can apply filters while fetching logs, e.g.,
- removing empty input: `meta.summary.input_text_length_i>0`
- fetching logs generated after a timestamp: `response_timestamp>=2018-09-18`

See more examples in [Logs notebook](https://github.com/watson-developer-cloud/assistant-improve-recommendations-notebook/blob/master/notebook/Logs%20Notebook.ipynb).

Note that logs will be saved into a cached file and loaded from the file. Set `overwrite` to True to refresh the cached file.

In [ ]:
# Define output filename
filename = 'logs'
# Create file name
if assistant_information['workspace_id'] is not None and len(assistant_information['workspace_id']) > 0:
    filename += '_workspace_' + assistant_information['workspace_id']
if assistant_information['assistant_id'] is not None and len(assistant_information['assistant_id']) > 0:
    filename += '_assistant_' + assistant_information['assistant_id']
if assistant_information['skill_id'] is not None and len(assistant_information['skill_id']) > 0:
    filename += '_skill_' +  assistant_information['skill_id']
# Remove all special characters from file name
filename = re.sub(r'[^a-zA-Z0-9_\- .]', '', filename) + '.json'

# Filter to be applied while fetching logs
filters = ['language::en',
           'meta.summary.input_text_length_i>0',
           'response_timestamp>=2020-01-01']

# Fetch the logs, set `overwrite` to True to reload logs
log_raw_data = get_logs(sdk_object,
                assistant_information,
                num_logs=20000,
                filename=filename,
                filters=filters,
                overwrite=True)

df_logs = pd.DataFrame(log_raw_data)

if log_raw_data is not None:
    # Mark that logs have been loaded
    logs_loaded = True
else:
    logs_loaded = False

### <a id="load_local"></a> 2.2 Option two: from JSON files

#### 2.2.1 Load assistant definition

You can use Watson Assistant `/workspaces` API to generate assistant definition file. See [Workspaces API](https://cloud.ibm.com/apidocs/assistant/assistant-v1#get-information-about-a-workspace) for more information. You can also use [Logs Notebook](http://) to prepare the assistant definition file.

In [8]:
if not assistant_loaded:
    # Specify a workspace JSON file
    assistant_definition_file = 'workspace.json'
    print('Loading workspace data from {}'.format(assistant_definition_file))

    # Store the workspace details in a dataframe
    df_assistant = json_normalize(json.load(open(assistant_definition_file)))

    # Get all intents
    assistant_intents = [intent['intent'] for intent in df_assistant['intents'].values[0]] 

    # Get all dialog nodes
    assistant_nodes = pd.DataFrame(df_assistant['dialog_nodes'].values[0])
    print('completed!')
else:
    print('Assistant definition has been loaded in Section 2.1.2.')

Assistant definition has been loaded in Section 2.1.2.


#### 2.2.2 Load a log JSON file

Another option is to load an existing log JSON file.  Log JSON files can be produced by using [Logs notebook](https://github.com/watson-developer-cloud/assistant-improve-recommendations-notebook/blob/master/notebook/Logs%20Notebook.ipynb), or [`fetch_logs`](https://github.com/watson-developer-cloud/assistant-improve-recommendations-notebook/blob/master/src/main/python/fetch_logs.py).

In [9]:
if not logs_loaded:
    
    # The following code is for loading your log file
    # Specify a log JSON file
    logs = load_logs_from_file(filename='logs.json')

    df_logs = pd.DataFrame(logs)
    print('completed!')
else:
    print('Logs have been loaded in Section 2.1.3.')

Logs have been loaded in Section 2.1.3.


### <a id="format_data"></a> 2.3 Format the log data
The logs returned from `logs` API are stored in a nested structure. In this step, we expand the nested structure and extract the fields used for analysis.

In [10]:
df_formatted = format_logs_disambiguation(df_logs)

Extracting request and response ...
Extracting response input ...
Extracting response output and context ...
Extracting disambiguation traffic ...
Extracting more options traffic ...
Completed!


## 3. Visualize Customer Effort <a id="overview"></a>

### 3.1 Extract utterances containing disambiguations or more options <a id="conversation_extraction"></a>

In [11]:
disambiguation_utterances = extract_disambiguation_utterances(df_formatted)

Extracting disambiguation logs from 11 conversations ...





Data Statistics:


Utterance,Count,Percentage
Total,6594,100.0%
Disambiguation,2301,34.9%
More Options,300,4.5%
Both,1114,16.9%
Conversation,Count,Percentage
Total,11,100.0%
Disambiguation,0,0.0%
More Options,0,0.0%
Both,11,100.0%


### 3.2 Calculate customer effort <a id="calculate"></a>
__Customer Effort__ quantifies how much effort your customers must make in order to achieve their intents. Example below illustrates how we calculate the customer effort of an utterance triggering both disambiguation and more options.

![Effort Calculation](image/effort_computation.png)

In [12]:
calculate_effort(disambiguation_utterances)

### 3.3 Visualize <a id="visualize"></a>

The figure below shows the average customer effort over different time intervals. `interval` to set the time interval among {"minute", "5-minute", "15-minute", "30-minute", "hour", "day", "week", "month"}. Move your cursor over the line or bars to check the effort value and distributions in detail. 

- Click Box Zoom icon (<img src="image/box_zoom.png" style="display:inline-block;vertical-align: bottom;" width="30" atl="Box Zoom"/>) or Wheel Zoom icon (<img src="image/wheel_zoom.png" style="display:inline-block;vertical-align: bottom;" width="25" atl="Wheel Zoom"/>) to zoom in or out the figure
- Click Reset icon (<img src="image/reset.png" style="display:inline-block;vertical-align: bottom;" width="25" atl="Reset"/>) to reset the view
- Click Save icon (<img src="image/save.png" style="display:inline-block;vertical-align: bottom;" width="25" atl="Save"/>) to save the figure in PNG format
- Click Autolearning Enabled Period button below the figure to view the area with Autolearning model applied



In [13]:
show_effort_over_time(disambiguation_utterances, interval='5-minute')

## 4. Analyze Customer Effort <a id="analysis"></a>

### 4.1 Customer effort vs user clicks <a id="effort_click"></a>

The figure below shows the total customer effort (red line) compared with user click distributions (bars) over different time intervals. Use `interval` to set the time interval among {"minute", "5-minute", "15-minute", "30-minute", "hour", "day", "week", "month"}. Move your cursor over the line or bars to check the effort value and distributions in detail. Click the entries in legend to show or hide the corresponding glyph. Click Autolearning Enabled Period button below the figure to view the area with Autolearning model applied.

Autolearning gains insights from user clicks from disambiguation and more options. As time goes on, autolearning reduces the uncertainty of your assistant when interacting with your customers. You should observe a decreasing trend of total customer effort over time. You may also observe a similar trend of user clicks (bars). These indicate autolearning has effectively improved your assistant.

In addition to reducing the uncertainty, autolearning also learns to include and promote the best answers in disambiguation and more options lists. Remember that clicking on "None of the Above" generates the highest customer effort, followed by More Options and Disambiguation. Therefore, you may observe an increasing proportion of disambiguation clicks (grey bar) with a decreasing proportion of More Options clicks (light blue bar) and None of the Above clicks (dark blue bar).


In [14]:
show_click_vs_effort(disambiguation_utterances, interval='5-minute')

### 4.2 Top dialog nodes with the highest effort<a id="effort_high"></a>

The figure below shows the top N nodes that generate the highest customer effort. Update the top variable specify the maximum number of nodes to visualize. Move the cursor on the bar to check number of logs and total customer effort. The X-Axis represents the total customer effort.

In [15]:
show_top_node_effort(disambiguation_utterances, top=10, assistant_nodes=assistant_nodes)

### 4.3 Customer effort on dialog nodes<a id="effort_node"></a>

- *\"How does customer effort change over time for specific dialog node?\"*
- *\"Does autolearning help on reducing the customer effort?\"*
- *\"What are the dialog nodes require additional analysis?\"*

The figure below helps you answer the above questions. The drop-down list contains all of the dialog nodes generated customer effort. Select a node and view the customer effort spent over time. Update the interval parameter to set a time interval. You can choose from:  {"minute", "5-minute", "15-minute", "30-minute", "hour", "day", "week", "month"}. Move your cursor over the bars to check the effort value.

In [16]:
show_node_effort(disambiguation_utterances, assistant_nodes, interval='5-minute')

### 4.4 Customer effort on utterances <a id="effort_utterance"></a>

- *\"What are the questions require high customer effort?\"*
- *\"Does autolearning help on reducing the customer effort?\"*
- *\"What are the questions require additional analysis?\"*

The figure below helps you answer the above questions. Use the drop-down to select an utterance to view its average customer effort over time. Use the top parameter to specify the top N utterances with the highest average effort. Use the interval parameter to set a time interval.  You can choose from: {"minute", "5-minute", "15-minute", "30-minute", "hour", "day", "week", "month"}. Move your cursor over the bars to check the effort value.


In [17]:
show_input_effort(disambiguation_utterances, top=20, interval='5-minute')

## 5. Identify the most confused dialog nodes<a id="node"></a>

Your assistant uses dialog nodes to interact with your customers. When there is no decisive dialog node that can handle a customer's input, your assistant triggers [Disambiguation](https://cloud.ibm.com/docs/assistant?topic=assistant-dialog-runtime#dialog-runtime-disambiguation) or More Options (TBD link) lists. These lists of potential nodes allow the customers to pick the right one. The Dialog nodes that frequently co-occur in Disambiguation or More Options are the most confused ones.


### 5.1 Top N frequently co-occurred dialog nodes<a id="node_top"></a>

The table below shows the top N dialog node pairs that frequently co-occurred in disambiguation and more options. These pairs of nodes cause confusion to your assistant. To eliminate confusion, check the intents used in each pair of nodes to make sure they are distinct. You can alleviate the problem using the following techniques:

1. Fix existing training that is causing confusion between intents
2. Add training to confused intents to clarify their boundaries
3. Add training to imprecise intents
4. Combine the confused intents into a single intent and distinguish using entities

Check "*Improve – Effectiveness*" on Page 9 of [Best Practices document](https://github.com/watson-developer-cloud/assistant-improve-recommendations-notebook/raw/master/notebook/IBM%20Watson%20Assistant%20Continuous%20Improvement%20Best%20Practices.pdf) for more information.

In [18]:
cooccurrence_matrix = generate_cooccurrence_matrix(disambiguation_utterances, assistant_nodes = assistant_nodes)
top_confused_pairs = cooccurrence_matrix.where(np.triu(np.ones(cooccurrence_matrix.shape)).astype(np.bool)).stack().nlargest(cooccurrence_matrix.shape[0]).to_frame()
top_confused_pairs.columns = ['Count']
top_confused_pairs = top_confused_pairs.reset_index()
top_confused_pairs.columns = ['Node A', 'Node B', 'Count']
show_cooccured_nodes(top_confused_pairs)


### 5.2 Dialog node co-occurrence heatmap <a id="node_heat"></a>

The heatmap below shows the co-occurrence count of the top N node pairs in disambiguation lists. Use `TOP_N_NODE_PAIR` to specify the number of pairs to visualize. You can move your cursor on each square to view count information.

In [19]:
# Select nodes appearing in the top co-occurred node pair
TOP_N_NODE_PAIR = 30
selected_nodes = pd.unique(top_confused_pairs.head(TOP_N_NODE_PAIR)[['Node A', 'Node B']].values.ravel())
selected_matrix = cooccurrence_matrix[selected_nodes].loc[selected_nodes]

# Select all nodes
# selected_matrix = cooccurrence_matrix

show_cooccured_heatmap(selected_matrix)


## 6. Analyze user clicks <a id="click"></a>

The figures below help you analyze the improvement autolearning made through user click data. "Click-1" indicates that users click on the top answer in Disambiguation or More Options. For example,

<img src="image/click.png" alt="User Clicks" width="400"/>

Remember that by observing and learning customer behavior, autolearning is able to (1) reduce the uncertainty of your assistant when interacting with your customers and (2) append and promote the best answers in disambiguation and more options lists.

Indicators for (1):
- A decreasing trend of clicks in both Disambiguation and More Options
- A more rapid reduction of clicks in More Options compared to Disambiguation

Indicators for (2):
- A reduction in the proportion of high-ranking clicks, e.g., "Click-5" and "Click-4", for both disambiguation and more options
- An increase in the proportion of "Click-1"

Use the interval parameter to set a time interval.  You can choose from: {"minute", "5-minute", "15-minute", "30-minute", "hour", "day", "week", "month"}. Move your cursor over bars to check click distributions in detail. Click the entries in legend to show or hide clicks.


In [20]:
show_disambiguation_click(disambiguation_utterances, interval='5-minute')

In [21]:
show_more_options_click(disambiguation_utterances, interval='5-minute')

Copyright © 2020 IBM. This notebook and its source code are released under the terms of the MIT License.